In [1]:
import sys
sys.path.insert(0, '..')
import cProfile
import numpy as np; print('numpy', np.__version__)
import zarr; print('zarr', zarr.__version__, 'blosc', ' '.join(zarr.blosc_version()))
import bcolz; print('bcolz', bcolz.__version__, 'blosc', ' '.join(bcolz.blosc_version()))
bcolz.blosc_set_nthreads(1)

numpy 1.10.2
zarr 0.1.1.dev6+dirty blosc 1.7.0 $Date:: 2015-07-05 #$
bcolz 0.12.2.dev22+dirty blosc 1.7.0 $Date:: 2015-07-05 #$


4

## Array creation

In [2]:
a = np.arange(1e8, dtype='i4')
a

array([       0,        1,        2, ..., 99999997, 99999998, 99999999], dtype=int32)

In [3]:
c = bcolz.carray(a, cparams=bcolz.cparams(cname='lz4'))
c

carray((100000000,), int32)
  nbytes: 381.47 MB; cbytes: 7.68 MB; ratio: 49.67
  cparams := cparams(clevel=5, shuffle=True, cname='lz4')
[       0        1        2 ..., 99999997 99999998 99999999]

In [4]:
z = zarr.array(a, chunks=c.chunklen, cname='lz4', synchronized=False)
z

zarr.ext.Array((100000000,), int32, chunks=(262144,), nbytes=381.5M, cbytes=6.6M, cratio=57.4, cname=lz4, clevel=5, shuffle=1)

In [5]:
%timeit bcolz.carray(a, cparams=bcolz.cparams(cname='lz4'))

10 loops, best of 3: 132 ms per loop


In [6]:
%timeit zarr.array(a, chunks=c.chunklen, cname='lz4')

10 loops, best of 3: 137 ms per loop


In [7]:
cProfile.run('zarr.array(a, chunks=c.chunklen, cname="lz4")', sort='time')

         8037 function calls (6888 primitive calls) in 0.146 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      382    0.133    0.000    0.133    0.000 ext.pyx:181(compress)
  762/382    0.003    0.000    0.137    0.000 ext.pyx:279(__setitem__)
    381/0    0.002    0.000    0.000          ext.pyx:526(genexpr)
      382    0.001    0.000    0.137    0.000 ext.pyx:154(__setitem__)
    381/0    0.001    0.000    0.000          ext.pyx:540(genexpr)
      763    0.001    0.000    0.002    0.000 numeric.py:1970(isscalar)
      381    0.001    0.000    0.001    0.000 {built-in method array}
      382    0.001    0.000    0.001    0.000 ext.pyx:109(is_total_slice)
      763    0.001    0.000    0.001    0.000 {built-in method isinstance}
      2/1    0.000    0.000    0.002    0.002 ext.pyx:432(__cinit__)
      382    0.000    0.000    0.001    0.000 ext.pyx:468(create_chunk)
      381    0.000    0.000    0.001    0.000 numeri

## Array read

In [8]:
%timeit c[:]

10 loops, best of 3: 180 ms per loop


In [9]:
%timeit z[:]

1 loops, best of 3: 226 ms per loop


In [10]:
cProfile.run('z[:]', sort='time')

         774 function calls (772 primitive calls) in 0.245 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      382    0.145    0.000    0.145    0.000 ext.pyx:242(decompress)
        1    0.095    0.095    0.242    0.242 ext.pyx:382(array_getitem)
      382    0.003    0.000    0.147    0.000 ext.pyx:221(__getitem__)
      2/1    0.002    0.001    0.245    0.245 <string>:1(<module>)
      2/1    0.000    0.000    0.245    0.245 {built-in method exec}
        1    0.000    0.000    0.000    0.000 ext.pyx:347(get_chunk_range)
        1    0.000    0.000    0.000    0.000 ext.pyx:287(normalise_array_selection)
        1    0.000    0.000    0.242    0.242 ext.pyx:501(__getitem__)
        1    0.000    0.000    0.000    0.000 ext.pyx:316(normalise_axis_selection)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




In [11]:
import line_profiler

In [12]:
profile = line_profiler.LineProfiler(zarr.ext.array_getitem)
profile.run('z[:]')
profile.print_stats()

Timer unit: 1e-06 s

Total time: 0.233539 s
File: /media/aliman/SD1/src/github/alimanfoo/zarr/zarr/ext.pyx
Function: array_getitem at line 382

Line #      Hits         Time  Per Hit   % Time  Line Contents
   382                                           def array_getitem(Array self, item):
   383                                               """Array.__getitem__ broken out as separate function to enable line
   384                                               profiling."""
   385                                           
   386                                               # normalise selection
   387         1           46     46.0      0.0      selection = normalise_array_selection(item, self.shape)
   388                                           
   389                                               # determine output array shape
   390         1           10     10.0      0.0      out_shape = tuple(stop - start for start, stop in selection)
   391                               